# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 8

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task8`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [ ]:
!az login

## Step 1. Write the code

In [1]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

Preparing Q# environment...


In [2]:
teamname="schrodingers-duck"  # Update this field with your team name
task=["task8"]
slack_id="U04L3SFUCQ2"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [3]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task8_DumpMachineWrapper : qsharp.QSharpCallable = None
Task8_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 8 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [4]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Synthesis;
open Microsoft.Quantum.Arrays;

// Task 8. 
// (input will contain 7 qubits)
operation original(input : Qubit[], target : Qubit) : Unit is Adj {
    for i in [15,23,27,29,30,31,39,43, 45,46,47,51,53,54,55,57, 
              58,59,60,61,62,63,71,75, 77,78,79,83,85,86,87,89,
              90,91,92,93,94,95,99,101, 102,103,105,106,107,108,109,110,
              111,113,114,115,116,117,118,119, 120,121,122,123,124,125,126,127] {
        ControlledOnInt(i, X)(input, target);
    }
}

operation Task8(input : Qubit[], target : Qubit) : Unit is Adj {
    use sumWires = Qubit[7];
    use zero = Qubit[1];
    within{
    //Ebene 1
    CarryN(input[0], input[1], input[2], sumWires[0]);
    CarryN(input[3], input[4], input[5], sumWires[1]);
    //Ebene 2
    CarryN(zero[0], input[2], input[5], sumWires[2]);
    CarryN(sumWires[2], sumWires[0], sumWires[1], sumWires[3]);
    //Ebene 3
    CarryN(zero[0], input[6], input[5], sumWires[4]);
    CarryN(sumWires[4], zero[0], sumWires[1], sumWires[5]);
    CarryN(sumWires[5], zero[0], sumWires[3], sumWires[6]);
    } apply{
    //CNOT(sumWires[3], target);
   CNOT(sumWires[3], target);
    }
}
operation CarryN(carryIn: Qubit, summand1: Qubit, summand2: Qubit, carryOut: Qubit) : Unit is Adj + Ctl {
    CCNOT (summand1, summand2, carryOut);
    CNOT (summand1, summand2);
    CCNOT (carryIn, summand2, carryOut);
    CNOT(carryIn, summand2);
}

operation QubitArrayWrapperOperation (op : ((Qubit[], Qubit) => Unit is Adj), qs : Qubit[]) : Unit is Adj {        
    op(Most(qs), Tail(qs));
}

operation TestCompoundGate () : Unit {
    AssertOperationsEqualReferenced(7+1, QubitArrayWrapperOperation(Task8, _), QubitArrayWrapperOperation (original, _));
}



In [5]:
TestCompoundGate();

In [6]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task8_DumpMachineWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task8(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task8_ResourceEstimationWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    Task8(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [7]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
qsharp.config["dump.truncateSmallAmplitudes"]=True
Task8_DumpMachineWrapper.simulate()

|00000000⟩	0.0883883476483185 + 0𝑖
|00000010⟩	0.0883883476483185 + 0𝑖
|00000100⟩	0.0883883476483185 + 0𝑖
|00000110⟩	0.0883883476483185 + 0𝑖
|00001000⟩	0.0883883476483185 + 0𝑖
|00001010⟩	0.0883883476483185 + 0𝑖
|00001100⟩	0.0883883476483185 + 0𝑖
|00001110⟩	0.0883883476483185 + 0𝑖
|00010000⟩	0.0883883476483185 + 0𝑖
|00010010⟩	0.0883883476483185 + 0𝑖
|00010100⟩	0.0883883476483185 + 0𝑖
|00010110⟩	0.0883883476483185 + 0𝑖
|00011000⟩	0.0883883476483185 + 0𝑖
|00011010⟩	0.0883883476483185 + 0𝑖
|00011100⟩	0.0883883476483185 + 0𝑖
|0001111100000000⟩	0.0883883476483185 + 0𝑖
|00100000⟩	0.0883883476483185 + 0𝑖
|00100010⟩	0.0883883476483185 + 0𝑖
|00100100⟩	0.0883883476483185 + 0𝑖
|00100110⟩	0.0883883476483185 + 0𝑖
|00101000⟩	0.0883883476483185 + 0𝑖
|00101010⟩	0.0883883476483185 + 0𝑖
|00101100⟩	0.0883883476483185 + 0𝑖
|0010111100000000⟩	0.0883883476483185 + 0𝑖
|00110000⟩	0.0883883476483185 + 0𝑖
|00110010⟩	0.0883883476483185 + 0𝑖
|00110100⟩	0.0883883476483185 + 0𝑖
|0011011100000000⟩	0.0883883476483185 + 0𝑖
|00111000⟩	0.0883883476483185 + 0𝑖
|0011101100000000⟩	0.0883883476483185 + 0𝑖
|0011110100000000⟩	0.0883883476483185 + 0𝑖
|0011111100000000⟩	0.0883883476483185 + 0𝑖
|01000000⟩	0.0883883476483185 + 0𝑖
|01000010⟩	0.0883883476483185 + 0𝑖
|01000100⟩	0.0883883476483185 + 0𝑖
|01000110⟩	0.0883883476483185 + 0𝑖
|01001000⟩	0.0883883476483185 + 0𝑖
|01001010⟩	0.0883883476483185 + 0𝑖
|01001100⟩	0.0883883476483185 + 0𝑖
|0100111100000000⟩	0.0883883476483185 + 0𝑖
|01010000⟩	0.0883883476483185 + 0𝑖
|01010010⟩	0.0883883476483185 + 0𝑖
|01010100⟩	0.0883883476483185 + 0𝑖
|0101011100000000⟩	0.0883883476483185 + 0𝑖
|01011000⟩	0.0883883476483185 + 0𝑖
|0101101100000000⟩	0.0883883476483185 + 0𝑖
|0101110100000000⟩	0.0883883476483185 + 0𝑖
|0101111100000000⟩	0.0883883476483185 + 0𝑖
|01100000⟩	0.0883883476483185 + 0𝑖
|01100010⟩	0.0883883476483185 + 0𝑖
|01100100⟩	0.0883883476483185 + 0𝑖
|0110011100000000⟩	0.0883883476483185 + 0𝑖
|01101000⟩	0.0883883476483185 + 0𝑖
|0110101100000000⟩	0.0883883476483185 + 0𝑖
|0110110100000000⟩	0.0883883476483185 + 0𝑖
|0110111100000000⟩	0.0883883476483185 + 0𝑖
|01110000⟩	0.0883883476483185 + 0𝑖
|0111001100000000⟩	0.0883883476483185 + 0𝑖
|0111010100000000⟩	0.0883883476483185 + 0𝑖
|0111011100000000⟩	0.0883883476483185 + 0𝑖
|0111100100000000⟩	0.0883883476483185 + 0𝑖
|0111101100000000⟩	0.0883883476483185 + 0𝑖
|0111110100000000⟩	0.0883883476483185 + 0𝑖
|0111111100000000⟩	0.0883883476483185 + 0𝑖
|10000000⟩	0.0883883476483185 + 0𝑖
|10000010⟩	0.0883883476483185 + 0𝑖
|10000100⟩	0.0883883476483185 + 0𝑖
|10000110⟩	0.0883883476483185 + 0𝑖
|10001000⟩	0.0883883476483185 + 0𝑖
|10001010⟩	0.0883883476483185 + 0𝑖
|10001100⟩	0.0883883476483185 + 0𝑖
|1000111100000000⟩	0.0883883476483185 + 0𝑖
|10010000⟩	0.0883883476483185 + 0𝑖
|10010010⟩	0.0883883476483185 + 0𝑖
|10010100⟩	0.0883883476483185 + 0𝑖
|1001011100000000⟩	0.0883883476483185 + 0𝑖
|10011000⟩	0.0883883476483185 + 0𝑖
|1001101100000000⟩	0.0883883476483185 + 0𝑖
|1001110100000000⟩	0.0883883476483185 + 0𝑖
|1001111100000000⟩	0.0883883476483185 + 0𝑖
|10100000⟩	0.0883883476483185 + 0𝑖
|10100010⟩	0.0883883476483185 + 0𝑖
|10100100⟩	0.0883883476483185 + 0𝑖
|1010011100000000⟩	0.0883883476483185 + 0𝑖
|10101000⟩	0.0883883476483185 + 0𝑖
|1010101100000000⟩	0.0883883476483185 + 0𝑖
|1010110100000000⟩	0.0883883476483185 + 0𝑖
|1010111100000000⟩	0.0883883476483185 + 0𝑖
|10110000⟩	0.0883883476483185 + 0𝑖
|1011001100000000⟩	0.0883883476483185 + 0𝑖
|1011010100000000⟩	0.0883883476483185 + 0𝑖
|1011011100000000⟩	0.0883883476483185 + 0𝑖
|1011100100000000⟩	0.0883883476483185 + 0𝑖
|1011101100000000⟩	0.0883883476483185 + 0𝑖
|1011110100000000⟩	0.0883883476483185 + 0𝑖
|1011111100000000⟩	0.0883883476483185 + 0𝑖
|11000000⟩	0.0883883476483185 + 0𝑖
|11000010⟩	0.0883883476483185 + 0𝑖
|11000100⟩	0.0883883476483185 + 0𝑖
|1100011100000000⟩	0.0883883476483185 + 0𝑖
|11001000⟩	0.0883883476483185 + 0𝑖
|1100101100000000⟩	0.0883883476483185 + 0𝑖
|1100110100000000⟩	0.0883883476483185 + 0𝑖
|1100111100000000⟩	0.0883883476483185 + 0𝑖
|11010000⟩	0.08838834764

()

## Step 3. Evaluate the code using resource estimation

In [9]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
qsharp.azure.connect(
    resourceId="/subscriptions/7fc54e95-14cf-4a6f-b50e-d6155c241309/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/azure-q",
    location="UK South",
    credential="CLI")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace azure-q in location uksouth.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 296609},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 366858},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 3},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 70},
 {'id': 'quantinuum.hqs-lt-s2-sim', 'current_availability': {}, 'average_queue_time': 183},
 {'id': 'quantinuum.hqs-lt', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current_availa

In [10]:
qsharp.azure.target("microsoft.estimator")

Loading package Microsoft.Quantum.Providers.Core and dependencies...
Active target is now microsoft.estimator


{'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0}

In [11]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task8_ResourceEstimationWrapper, jobName="RE for the task 8")

Submitting Task8_ResourceEstimationWrapper to target microsoft.estimator...
Job successfully submitted.
   Job name: RE for the task 8
   Job ID: e310d987-293a-4420-8db8-d09b15c2f5ac
Waiting up to 30 seconds for Azure Quantum job to complete...
[8:04:56 AM] Current job status: Succeeded


In [12]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

Physical qubits,108090,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 10890 physical qubits to implement the algorithm logic, and 97200 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Runtime,369us 600ns,"Total runtime This is a runtime estimate (in nanosecond precision) for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (4us 400ns) multiplied by the 84 logical cycles to run the algorithm. If however the duration of a single T factory (here: 46us 800ns) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
Logical algorithmic qubits,45,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the \(Q_{\rm alg} = 16\) logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 45$ logical qubits."
Algorithmic depth,84,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 0 single-qubit measurements, the 0 arbitrary single-qubit rotations, and the 0 T gates, three multi-qubit measurements for each of the 28 CCZ and 0 CCiX gates in the input program, as well as No rotations in algorithm multi-qubit measurements for each of the 0 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,84,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 84. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Number of T states,112,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 0 T gates, four T states for each of the 28 CCZ and 0 CCiX gates, as well as No rotations in algorithm for each of the 0 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,15,Number of T factories capable of producing the demanded 112 T states during the algorithm's runtime The total number of T factories 15 that are executed in parallel is computed as \(\left\lceil\dfrac{112\;\text{T states} \cdot 46us 800ns\;\text{T factory duration}}{1\;\text{T states per T factory} \cdot 369us 600ns\;\text{algorithm runtime}}\right\rceil\)
Number of T factory invocations,8,"Number of times all T factories are invoked In order to prepare the 112 T states, the 15 copies of the T factory are repeatedly invoked 8 times."
Physical algorithmic qubits,10890,Number of physical qubits for the algorithm after layout The 10890 are the product of the 45 logical qubits after layout and the 242 physical qubits that encode a single logical qubit.
Physical T factory qubits,97200,"Number of physical qubits for the T factories Each T factory requires 6480 physical qubits and we run 15 in parallel, therefore we need $97200 = 6480 \cdot 15$ qubits."
Required logical qubit error rate,1.32e-7,The minimum logical qubit error rate required to run the algorithm within the error budget The minimum logical qubit error rate is obtained by dividing the logical error probability 5.00e-4 by the product of 45 logical qubits and the total cycle count 84.


In [13]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [14]:
evaluate_results(result)

Logical algorithmic qubits = 45
Algorithmic depth = 84
Score = 3780


3780